In [16]:
import cvxpy as cp
import numpy as np

In [17]:
cost = np.array([3,3,3]).reshape(-1,1)
demand = np.array([[1.5,1.5,1.5],[2,2,2],[3.5,3.5,3.5],[4,4,4],[5.5,5.5,5.5]]).reshape(-1,3)
produce = np.array([8,8,8]).reshape(-1,1)

In [18]:
def solve_second_stage_subproblem(produce,all_demand):

    selling_price = np.array([5,5,5]).reshape(-1,1)
    salvage_value = np.array([1,1,1]).reshape(-1,1)
    size = 3
    duals = []
    vs = []

    for demand in all_demand:

        shape = (size,1)

        sell = cp.Variable(shape=(size,1))
        salvage = cp.Variable(shape=(size,1))

        objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_value.T@salvage))
        constraints =  [sell+salvage<=produce,sell<=demand[0].reshape(-1,1),salvage>=np.zeros_like(salvage).reshape(-1,1),sell>=np.zeros_like(sell).reshape(-1,1)]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals.append(dual_value)
        vs.append(problem.value)
    
    return np.array(duals).reshape(-1,size),np.array(vs).reshape(-1,1)

In [19]:
duals,qs = solve_second_stage_subproblem(produce,demand)

In [20]:
def solve_first_stage_problem(produce,limit,g_ks,alpha_ks):
    
    size = 3
    cost = np.array([3,3,3]).reshape(-1,1)

    produce  = cp.Variable(shape=(3,1))
    v = cp.Variable(shape=(1,1))

    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[i]).T@produce+np.array(alpha_ks[i])<=v)

    constraints.extend([produce>=0,v>=-1000000,produce>=np.full((size,1),limit),produce<=np.full((size,1),100)])

    objective = cp.Minimize((cost.T@produce)+v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return produce.value, v.value

In [21]:
def solve_first_stage_problem_2(produce,g_ks,alpha_ks):
    
    size = 3
    cost = np.array([3,3,3]).reshape(-1,1)

    produce  = cp.Variable(shape=(3,1))
    v = cp.Variable(shape=(1,1))

    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[i]).T@produce+np.array(alpha_ks[i])<=v)

    constraints.extend([produce>=0,v>=-1000000,produce<=np.full((size,1),100)])

    objective = cp.Minimize((cost.T@produce)+v)

    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.value)

    return produce.value, v.value

In [22]:
produce = np.array([3,3,3]).reshape(-1,1)
limit = -100000

g_ks = []
alpha_ks = []

for i in range(0,100):
    duals,qs = solve_second_stage_subproblem(produce,demand)

    senario = 5
    senario_prob = (1/senario)*(np.ones_like(demand[:,0])).reshape(-1,1)
    g_ks_temp = (-senario_prob.T@duals).T
    alpha_ks_temp = senario_prob.T@qs - g_ks_temp.T@produce

    g_ks.append(g_ks_temp)
    alpha_ks.append(alpha_ks_temp)

    produce,limit= solve_first_stage_problem_2(produce,g_ks,alpha_ks)
    print(produce)
    # print(limit)

    


-128.40000031616682
[[99.99999994]
 [99.99999994]
 [99.99999994]]
-13.59999997415283
[[4.33333333]
 [4.33333333]
 [4.33333333]]
-12.899999973628958
[[3.74999999]
 [3.74999999]
 [3.74999999]]
-12.599999973686103
[[3.5]
 [3.5]
 [3.5]]
-12.599999962837963
[[3.50000001]
 [3.5       ]
 [3.5       ]]
-12.5999999759699
[[1.35498647]
 [4.89513153]
 [4.24988199]]
-12.599999981953594
[[3.19452915]
 [5.56227015]
 [1.74320068]]
-12.59999998212033
[[2.96390997]
 [2.04343488]
 [5.49265513]]
-12.599999994918953
[[2.98537344]
 [4.43637705]
 [3.07824947]]
-12.599999987254208
[[3.36080266]
 [3.72828733]
 [3.41090999]]
-12.599999984595954
[[3.33152011]
 [3.48891957]
 [3.6795603 ]]
-12.599999985830099
[[3.50000234]
 [3.49999898]
 [3.49999864]]
-12.599999982404832
[[3.49999995]
 [3.5       ]
 [3.50000001]]
-12.599999985323876
[[3.49999997]
 [3.5       ]
 [3.50000001]]
-12.59999998549047
[[3.49999996]
 [3.5       ]
 [3.5       ]]
-12.59999998456334
[[3.49999997]
 [3.5       ]
 [3.49999999]]
-12.599999980636

In [19]:
a =np.array([[0], [0]])

In [20]:
a.shape

(2, 1)